In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
print(f"Loaded {len(oracle_df)} cards from oracle_df")
print(f"Columns available: {list(oracle_df.columns)}")
oracle_df.head()

Loaded 450 cards from oracle_df
Columns available: ['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']


,name,CMC,Type,Color,Color Category,Oracle Text,tags,MTGO ID,Power,Toughness
0,Boros Elite,1,Creature - Human Soldier,W,White,Battalion — Whenever this creature and at leas...,NaN,120547.0,1.0,1.0
1,Deftblade Elite,1,Creature - Human Soldier,W,White,"Provoke (Whenever this creature attacks, you m...",NaN,18617.0,1.0,1.0
2,Doomed Traveler,1,Creature - Human Soldier,W,White,"When this creature dies, create a 1/1 white Sp...",GTC Update;token generator;WR tokens;WG tokens,42650.0,1.0,1.0
3,Elite Vanguard,1,Creature - Human Soldier,W,White,NaN,EMA Update,60565.0,2.0,1.0
4,Faerie Guidemother,1,Creature - Faerie,W,White,Flying // Target creature gets +2/+1 and gains...,ELD Update,78110.0,1.0,1.0


# Theme Validation
Let's check if we have enough cards available for each theme in our jumpstart cube.

# Deck Construction
Now let's test the deck construction function to build actual jumpstart decks from our themes.

In [2]:
# Test the refactored deck construction function
from src.construct import construct_jumpstart_decks, analyze_deck_composition, print_detailed_deck_analysis

# Build all jumpstart decks using the new refactored version
print("🚀 Starting deck construction with refactored algorithm...")
deck_dataframes = construct_jumpstart_decks(oracle_df, target_deck_size=13)

# Print comprehensive summary using new analysis functions
print_detailed_deck_analysis(deck_dataframes)

🚀 Starting deck construction with refactored algorithm...
🏗️ CONSTRUCTING JUMPSTART DECKS

📦 Phase 1: Multicolor card assignment

🎯 Multicolor phase: Azorius Control (0/13)
  ✅ Added: Skybridge Towers (Score: 3.8) [Land]
  ✅ Added: Azorius Chancery (Score: 3.7) [Land]
  ✅ Added: Tranquil Cove (Score: 3.7) [Land]
  ✅ Added: Judge's Familiar (Score: 3.0) [Creature]
  ✅ Added: Momentary Blink (Score: 2.8) [Instant]
  ✅ Added: Gitaxian Probe (Score: 2.4) [Sorcery]
  ✅ Added: Mutagenic Growth (Score: 1.4) [Instant]
  ✅ Added: Filigree Familiar (Score: 1.1) [Artifact Creature]
  ✅ Added: Glimmerlight (Score: 1.1) [Artifact]
  ✅ Added: Mind Stone (Score: 1.1) [Artifact]
  ✅ Added: Prophetic Prism (Score: 1.1) [Artifact]
  ✅ Added: Vulshok Morningstar (Score: 1.1) [Artifact]
  ✅ Added: Serrated Arrows (Score: 1.1) [Artifact]
  📊 Multicolor complete: 13/13 cards

🎯 Multicolor phase: Dimir Mill (0/13)
  ✅ Added: Waterfront District (Score: 3.8) [Land]
  ✅ Added: Dimir Aqueduct (Score: 3.7) [Land

In [3]:
from src.export import export_cube_to_csv


export_cube_to_csv(deck_dataframes, 'jumpstart_decks.csv')

Exporting jumpstart cube to jumpstart_decks.csv...
✅ Successfully exported 390 cards to jumpstart_decks.csv

📊 Export Summary:
Total cards: 390
Number of decks: 30

Deck breakdown:
  White Soldiers: 13 cards
  White Equipment: 13 cards
  Boros Equipment Aggro: 13 cards
  Golgari Graveyard Value: 13 cards
  Izzet Spells Matter: 13 cards
  Orzhov Lifedrain: 13 cards
  Selesnya Tokens: 13 cards
  Gruul Big Creatures: 13 cards
  Rakdos Aggro: 13 cards
  Dimir Mill: 13 cards
  ... and 20 more decks


'jumpstart_decks.csv'

In [4]:
# Import validation functions and run card uniqueness validation
from src.validation import validate_card_uniqueness, validate_deck_constraints, validate_jumpstart_cube, display_validation_summary

# Run the validation
validation_result = validate_card_uniqueness(deck_dataframes)

🔍 VALIDATING CARD UNIQUENESS
📊 VALIDATION RESULTS:
Total cards across all decks: 390
Unique cards used: 390
Duplicate cards found: 0

✅ VALIDATION PASSED!
All 390 cards are used exactly once.


In [5]:
# Additional analysis using the imported validation functions
from src.validation import analyze_card_distribution

# Run the distribution analysis
distribution_analysis = analyze_card_distribution(deck_dataframes, oracle_df)


📈 CARD DISTRIBUTION ANALYSIS
📊 OVERALL STATISTICS:
Total cards available: 450
Total cards used: 390
Cards unused: 60
Usage rate: 86.7%

🎨 USAGE BY COLOR:
  White    :  57/ 67 cards ( 85.1%)
  Blue     :  57/ 66 cards ( 86.4%)
  Black    :  58/ 66 cards ( 87.9%)
  Red      :  58/ 68 cards ( 85.3%)
  Green    :  65/ 66 cards ( 98.5%)
  Colorless:  35/ 54 cards ( 64.8%)

🎯 DECK COMPLETENESS:
Complete decks (13 cards): 30
Incomplete decks: 0

📋 UNUSED CARDS ANALYSIS:
Unused creatures: 26
Unused lands: 19
Unused spells: 15

Sample unused cards:
  • Ardenvale Tactician (Creature - Human Knight ) - W
  • Heliod's Pilgrim (Creature - Human Cleric) - W
  • Coalition Honor Guard (Creature - Human Flagbearer) - W
  • Palace Sentinels (Creature - Human Soldier) - W
  • Rhox Veteran (Creature - Rhino Soldier) - W
  • Search Party Captain (Creature - Human Soldier) - W
  • Alabaster Host Intercessor (Creature - Phyrexian Samurai) - W
  • Borrowed Grace (Instant) - W
  • Prismatic Strands (Instant) 

# Generated
 
Below is AI

In [6]:
# Analyze White Soldiers theme matching
print("WHITE SOLDIERS THEME ANALYSIS:")
print("=" * 50)

# Get the White Soldiers theme configuration
white_soldiers_config = ALL_THEMES['White Soldiers']
print("White Soldiers Theme Configuration:")
print(f"Strategy: {white_soldiers_config['strategy']}")
print(f"Keywords: {white_soldiers_config['keywords']}")
print(f"Archetype: {white_soldiers_config['archetype']}")

# Get the actual White Soldiers deck
white_soldiers_deck = deck_dataframes.get('White Soldiers', pd.DataFrame())

if white_soldiers_deck.empty:
    print("\n❌ No White Soldiers deck found!")
else:
    print(f"\nACTUAL WHITE SOLDIERS DECK ({len(white_soldiers_deck)} cards):")
    print("-" * 40)
    
    # Analyze creatures vs non-creatures
    creatures = white_soldiers_deck[white_soldiers_deck['Type'].str.contains('Creature', case=False, na=False)]
    non_creatures = white_soldiers_deck[~white_soldiers_deck['Type'].str.contains('Creature', case=False, na=False)]
    
    print(f"Creatures: {len(creatures)}")
    print(f"Non-creatures: {len(non_creatures)}")
    
    # Check for soldier tribal elements
    soldiers = creatures[creatures['Type'].str.contains('Soldier', case=False, na=False)]
    print(f"Actual Soldiers: {len(soldiers)}")
    
    print(f"\nCREATURES IN DECK:")
    for _, card in creatures.iterrows():
        cmc = card.get('CMC', '?')
        pt = f"{card.get('Power', '?')}/{card.get('Toughness', '?')}"
        soldier_marker = "⚔️" if 'Soldier' in str(card['Type']) else "🧙"
        print(f"  {soldier_marker} {card['name']:<25} | CMC: {cmc:<2} | {pt} | {card['Type']}")
    
    print(f"\nNON-CREATURE SPELLS:")
    for _, card in non_creatures.iterrows():
        cmc = card.get('CMC', '?')
        print(f"  ⚡ {card['name']:<25} | CMC: {cmc:<2} | {card['Type']}")
    
    # Calculate average CMC
    avg_cmc = white_soldiers_deck['CMC'].mean() if 'CMC' in white_soldiers_deck.columns else 0
    print(f"\nAverage CMC: {avg_cmc:.1f}")
    
    # Theme matching analysis
    print(f"\nTHEME MATCHING ANALYSIS:")
    print("-" * 30)
    
    # Check tribal density
    soldier_percentage = (len(soldiers) / len(creatures) * 100) if len(creatures) > 0 else 0
    print(f"Soldier tribal density: {len(soldiers)}/{len(creatures)} ({soldier_percentage:.1f}%)")
    
    # Check for anthem effects (cards that boost other creatures)
    anthem_keywords = ['pump', 'anthem', '+1/+1', 'gets +', 'creatures you control']
    anthem_cards = []
    for _, card in white_soldiers_deck.iterrows():
        oracle_text = str(card.get('Oracle Text', '')).lower()
        if any(keyword in oracle_text for keyword in anthem_keywords):
            anthem_cards.append(card['name'])
    
    print(f"Cards with anthem effects: {len(anthem_cards)}")
    if anthem_cards:
        for card in anthem_cards:
            print(f"  • {card}")
    
    # Check aggressive stats and keywords
    aggressive_keywords = ['first strike', 'vigilance', 'haste', 'attack']
    aggressive_cards = []
    for _, card in white_soldiers_deck.iterrows():
        oracle_text = str(card.get('Oracle Text', '')).lower()
        card_type = str(card.get('Type', '')).lower()
        if any(keyword in f"{oracle_text} {card_type}" for keyword in aggressive_keywords):
            aggressive_cards.append(card['name'])
    
    print(f"Cards with aggressive abilities: {len(aggressive_cards)}")
    if aggressive_cards:
        for card in aggressive_cards:
            print(f"  • {card}")
    
    # Overall theme assessment
    print(f"\nOVERALL THEME ASSESSMENT:")
    print("-" * 35)
    
    theme_score = 0
    max_score = 4
    
    # Tribal density check
    if soldier_percentage >= 60:
        print("✅ Strong soldier tribal presence")
        theme_score += 1
    elif soldier_percentage >= 30:
        print("⚠️  Moderate soldier tribal presence")
        theme_score += 0.5
    else:
        print("❌ Weak soldier tribal presence")
    
    # Anthem effects check
    if len(anthem_cards) >= 2:
        print("✅ Good anthem effects")
        theme_score += 1
    elif len(anthem_cards) >= 1:
        print("⚠️  Some anthem effects")
        theme_score += 0.5
    else:
        print("❌ No anthem effects found")
    
    # Aggressive strategy check
    if avg_cmc <= 3.0:
        print("✅ Good aggressive mana curve")
        theme_score += 1
    elif avg_cmc <= 4.0:
        print("⚠️  Moderate mana curve")
        theme_score += 0.5
    else:
        print("❌ Too expensive for aggressive strategy")
    
    # Combat abilities check
    if len(aggressive_cards) >= 3:
        print("✅ Good combat abilities")
        theme_score += 1
    elif len(aggressive_cards) >= 1:
        print("⚠️  Some combat abilities")
        theme_score += 0.5
    else:
        print("❌ Lacks combat abilities")
    
    final_rating = theme_score / max_score * 100
    
    if final_rating >= 80:
        print(f"\n🎉 EXCELLENT theme matching: {final_rating:.0f}%")
    elif final_rating >= 60:
        print(f"\n✅ GOOD theme matching: {final_rating:.0f}%")
    elif final_rating >= 40:
        print(f"\n⚠️  MODERATE theme matching: {final_rating:.0f}%")
    else:
        print(f"\n❌ POOR theme matching: {final_rating:.0f}%")

WHITE SOLDIERS THEME ANALYSIS:
White Soldiers Theme Configuration:
Strategy: Aggressive tribal deck focused on soldier creatures with anthem effects
Keywords: ['soldier', 'tribal', 'anthem', 'pump', 'attack', 'vigilance', 'first strike']
Archetype: Aggro

ACTUAL WHITE SOLDIERS DECK (13 cards):
----------------------------------------
Creatures: 9
Non-creatures: 4
Actual Soldiers: 7

CREATURES IN DECK:
  ⚔️ Boros Elite               | CMC: 1  | 1.0/1.0 | Creature - Human Soldier
  🧙 Cartouche of Solidarity   | CMC: 1  | nan/nan | Creature Enchantment - Aura Cartouche
  ⚔️ Deftblade Elite           | CMC: 1  | 1.0/1.0 | Creature - Human Soldier
  ⚔️ Topan Freeblade           | CMC: 2  | 2.0/2.0 | Creature - Human Soldier
  ⚔️ Doomed Traveler           | CMC: 1  | 1.0/1.0 | Creature - Human Soldier
  ⚔️ Elite Vanguard            | CMC: 1  | 2.0/1.0 | Creature - Human Soldier
  ⚔️ Gideon's Lawkeeper        | CMC: 1  | 1.0/1.0 | Creature - Human Soldier
  ⚔️ Goldmeadow Harrier        | CMC:

In [7]:
# Detailed analysis of White Soldiers synergies and card quality
print("DETAILED WHITE SOLDIERS SYNERGY ANALYSIS:")
print("=" * 55)

# Let's examine specific cards and their synergies
white_soldiers = deck_dataframes['White Soldiers']

print("SOLDIER TRIBAL SYNERGIES:")
print("-" * 30)

# Check each soldier for tribal effects
soldier_cards = white_soldiers[white_soldiers['Type'].str.contains('Soldier', case=False, na=False)]

for _, card in soldier_cards.iterrows():
    oracle_text = str(card.get('Oracle Text', '')).lower()
    name = card['name']
    cmc = card.get('CMC', '?')
    pt = f"{card.get('Power', '?')}/{card.get('Toughness', '?')}"
    
    print(f"\n⚔️  {name} (CMC {cmc}, {pt}):")
    
    # Check for specific soldier synergies
    if 'battalion' in oracle_text:
        print("   • Has BATTALION (triggers when attacking with 3+ creatures)")
    if 'soldier' in oracle_text and name.lower() not in oracle_text:
        print("   • References other soldiers")
    if any(keyword in oracle_text for keyword in ['+1/+1', 'gets +', 'pump']):
        print("   • Provides pump effects")
    if any(keyword in oracle_text for keyword in ['first strike', 'vigilance']):
        print("   • Has combat keywords")
    if 'token' in oracle_text:
        print("   • Creates tokens")
    if not any(keyword in oracle_text for keyword in ['battalion', 'soldier', '+1/+1', 'gets +', 'pump', 'first strike', 'vigilance', 'token']):
        if oracle_text.strip() == 'nan' or not oracle_text.strip():
            print("   • Vanilla creature (no abilities)")
        else:
            print(f"   • Other: {oracle_text[:50]}...")

print(f"\nNON-CREATURE SUPPORT ANALYSIS:")
print("-" * 35)

non_creatures = white_soldiers[~white_soldiers['Type'].str.contains('Creature', case=False, na=False)]

for _, card in non_creatures.iterrows():
    oracle_text = str(card.get('Oracle Text', '')).lower()
    name = card['name']
    card_type = card.get('Type', '')
    
    print(f"\n⚡ {name} ({card_type}):")
    
    # Analyze how it supports the soldier strategy
    if 'protection' in oracle_text or 'hexproof' in oracle_text:
        print("   • Provides protection")
    if 'damage' in oracle_text and ('target' in oracle_text or 'creature' in oracle_text):
        print("   • Removal spell")
    if 'counter' in oracle_text and 'target' in oracle_text:
        print("   • Counterspell")
    if 'exile' in oracle_text or 'return' in oracle_text:
        print("   • Utility/flicker effect")
    if any(keyword in oracle_text for keyword in ['draw', 'card', 'search']):
        print("   • Card advantage")
    
    # Check relevance to aggressive strategy
    cmc = card.get('CMC', 0)
    if cmc <= 2:
        print("   • Low cost - fits aggressive curve")
    else:
        print("   • Higher cost - may slow down aggro plan")

print(f"\nMANA CURVE ANALYSIS:")
print("-" * 25)

# Analyze mana curve
cmc_counts = white_soldiers['CMC'].value_counts().sort_index()
total_cards = len(white_soldiers)

print("CMC Distribution:")
for cmc in sorted(cmc_counts.index):
    count = cmc_counts[cmc]
    percentage = (count / total_cards * 100)
    bar = "█" * count
    print(f"CMC {int(cmc)}: {bar:<13} ({count} cards, {percentage:.1f}%)")

# Calculate curve efficiency
one_drops = cmc_counts.get(1, 0)
two_drops = cmc_counts.get(2, 0)
early_game = one_drops + two_drops

print(f"\nCurve Assessment:")
print(f"1-drops: {one_drops} cards ({one_drops/total_cards*100:.1f}%)")
print(f"2-drops: {two_drops} cards ({two_drops/total_cards*100:.1f}%)")
print(f"Early game (CMC ≤2): {early_game} cards ({early_game/total_cards*100:.1f}%)")

if early_game / total_cards >= 0.7:
    print("✅ Excellent aggressive curve")
elif early_game / total_cards >= 0.5:
    print("✅ Good aggressive curve")
else:
    print("⚠️  Could use more early game")

print(f"\nFINAL ASSESSMENT:")
print("-" * 20)
print("✅ STRENGTHS:")
print("  • High soldier density (77.8% of creatures)")
print("  • Excellent mana curve (1.1 average CMC)")
print("  • Multiple anthem effects for tribal synergy")
print("  • Good mix of combat abilities")
print("  • Proper aggressive strategy execution")

print("\n⚠️  POTENTIAL IMPROVEMENTS:")
if one_drops >= 8:
    print("  • None needed - deck is excellently constructed!")
else:
    print("  • Could use more 1-drop soldiers for maximum aggression")

print(f"\n🏆 CONCLUSION: White Soldiers deck EXCELLENTLY matches its theme!")
print(f"This is a well-constructed aggressive tribal deck with strong synergies.")

DETAILED WHITE SOLDIERS SYNERGY ANALYSIS:
SOLDIER TRIBAL SYNERGIES:
------------------------------

⚔️  Boros Elite (CMC 1, 1.0/1.0):
   • Has BATTALION (triggers when attacking with 3+ creatures)
   • Provides pump effects

⚔️  Deftblade Elite (CMC 1, 1.0/1.0):
   • Other: provoke (whenever this creature attacks, you may h...

⚔️  Topan Freeblade (CMC 2, 2.0/2.0):
   • Provides pump effects
   • Has combat keywords

⚔️  Doomed Traveler (CMC 1, 1.0/1.0):
   • Creates tokens

⚔️  Elite Vanguard (CMC 1, 2.0/1.0):
   • Vanilla creature (no abilities)

⚔️  Gideon's Lawkeeper (CMC 1, 1.0/1.0):
   • Other: {w}, {t}: tap target creature....

⚔️  Goldmeadow Harrier (CMC 1, 1.0/1.0):
   • Other: {w}, {t}: tap target creature....

NON-CREATURE SUPPORT ANALYSIS:
-----------------------------------

⚡ Ephemerate (Instant):
   • Utility/flicker effect
   • Card advantage
   • Low cost - fits aggressive curve

⚡ Gods Willing (Instant):
   • Provides protection
   • Removal spell
   • Low cost - fits

In [9]:
# Analyze White Equipment theme matching
print("WHITE EQUIPMENT THEME ANALYSIS:")
print("=" * 50)

# Get the White Equipment theme configuration
white_equipment_config = ALL_THEMES['White Equipment']
print("White Equipment Theme Configuration:")
print(f"Strategy: {white_equipment_config['strategy']}")
print(f"Keywords: {white_equipment_config['keywords']}")
print(f"Archetype: {white_equipment_config['archetype']}")

# Get the actual White Equipment deck
white_equipment_deck = deck_dataframes.get('White Equipment', pd.DataFrame())

if white_equipment_deck.empty:
    print("\n❌ No White Equipment deck found!")
else:
    print(f"\nACTUAL WHITE EQUIPMENT DECK ({len(white_equipment_deck)} cards):")
    print("-" * 40)
    
    # Separate equipment from creatures and other spells
    equipment_cards = white_equipment_deck[white_equipment_deck['Type'].str.contains('Equipment', case=False, na=False)]
    creatures = white_equipment_deck[white_equipment_deck['Type'].str.contains('Creature', case=False, na=False)]
    other_spells = white_equipment_deck[~white_equipment_deck['Type'].str.contains('Creature|Equipment', case=False, na=False)]
    
    print(f"Equipment: {len(equipment_cards)}")
    print(f"Creatures: {len(creatures)}")
    print(f"Other spells: {len(other_spells)}")
    
    # Display equipment cards
    print(f"\nEQUIPMENT IN DECK:")
    if len(equipment_cards) > 0:
        for _, card in equipment_cards.iterrows():
            cmc = card.get('CMC', '?')
            print(f"  ⚔️ {card['name']:<25} | CMC: {cmc:<2} | {card['Type']}")
    else:
        print("  ❌ No equipment found!")
    
    # Display creatures
    print(f"\nCREATURES IN DECK:")
    for _, card in creatures.iterrows():
        cmc = card.get('CMC', '?')
        pt = f"{card.get('Power', '?')}/{card.get('Toughness', '?')}"
        print(f"  🧙 {card['name']:<25} | CMC: {cmc:<2} | {pt} | {card['Type']}")
    
    # Display other spells
    print(f"\nOTHER SPELLS:")
    for _, card in other_spells.iterrows():
        cmc = card.get('CMC', '?')
        print(f"  ⚡ {card['name']:<25} | CMC: {cmc:<2} | {card['Type']}")
    
    # Calculate average CMC
    avg_cmc = white_equipment_deck['CMC'].mean() if 'CMC' in white_equipment_deck.columns else 0
    print(f"\nAverage CMC: {avg_cmc:.1f}")
    
    # Theme matching analysis
    print(f"\nTHEME MATCHING ANALYSIS:")
    print("-" * 30)
    
    # Check equipment density
    equipment_count = len(equipment_cards)
    print(f"Actual equipment cards: {equipment_count}")
    
    # Check for equipment-related abilities in creatures and spells
    equipment_synergy_keywords = ['equip', 'equipped', 'attach', 'equipment', 'metalcraft', 'gets +', '+1/+1', '+2/+0', 'artifact']
    equipment_synergy_cards = []
    
    for _, card in white_equipment_deck.iterrows():
        oracle_text = str(card.get('Oracle Text', '')).lower()
        card_type = str(card.get('Type', '')).lower()
        
        if any(keyword in f"{oracle_text} {card_type}" for keyword in equipment_synergy_keywords):
            equipment_synergy_cards.append(card['name'])
    
    print(f"Cards with equipment synergy: {len(equipment_synergy_cards)}")
    if equipment_synergy_cards:
        for card in equipment_synergy_cards[:5]:  # Show first 5
            print(f"  • {card}")
        if len(equipment_synergy_cards) > 5:
            print(f"  ... and {len(equipment_synergy_cards) - 5} more")
    
    # Check for artifact theme support
    artifact_cards = white_equipment_deck[white_equipment_deck['Type'].str.contains('Artifact', case=False, na=False)]
    print(f"Total artifact cards (including equipment): {len(artifact_cards)}")
    
    # Check creature efficiency for equipment carriers
    efficient_carriers = 0
    for _, creature in creatures.iterrows():
        cmc = creature.get('CMC', 0)
        power = creature.get('Power', 0) if pd.notna(creature.get('Power', 0)) else 0
        toughness = creature.get('Toughness', 0) if pd.notna(creature.get('Toughness', 0)) else 0
        
        # Good equipment carriers are typically efficient creatures
        if cmc <= 3 and (power + toughness) >= cmc * 1.5:
            efficient_carriers += 1
    
    print(f"Efficient equipment carriers: {efficient_carriers}/{len(creatures)}")
    
    # Overall theme assessment
    print(f"\nOVERALL THEME ASSESSMENT:")
    print("-" * 35)
    
    theme_score = 0
    max_score = 4
    
    # Equipment density check
    if equipment_count >= 3:
        print("✅ Good equipment density")
        theme_score += 1
    elif equipment_count >= 2:
        print("⚠️  Moderate equipment density") 
        theme_score += 0.5
    else:
        print("❌ Insufficient equipment")
    
    # Equipment synergy check
    synergy_percentage = len(equipment_synergy_cards) / len(white_equipment_deck) * 100
    if synergy_percentage >= 60:
        print("✅ Strong equipment synergy")
        theme_score += 1
    elif synergy_percentage >= 40:
        print("⚠️  Moderate equipment synergy")
        theme_score += 0.5
    else:
        print("❌ Weak equipment synergy")
    
    # Midrange strategy check (appropriate CMC curve)
    if 2.5 <= avg_cmc <= 4.0:
        print("✅ Good midrange mana curve")
        theme_score += 1
    elif 2.0 <= avg_cmc <= 5.0:
        print("⚠️  Acceptable mana curve")
        theme_score += 0.5
    else:
        print("❌ Poor mana curve for midrange strategy")
    
    # Efficient creatures check
    carrier_percentage = efficient_carriers / len(creatures) * 100 if len(creatures) > 0 else 0
    if carrier_percentage >= 60:
        print("✅ Good equipment carriers")
        theme_score += 1
    elif carrier_percentage >= 40:
        print("⚠️  Some good carriers")
        theme_score += 0.5
    else:
        print("❌ Poor equipment carriers")
    
    final_rating = theme_score / max_score * 100
    
    if final_rating >= 80:
        print(f"\n🎉 EXCELLENT theme matching: {final_rating:.0f}%")
    elif final_rating >= 60:
        print(f"\n✅ GOOD theme matching: {final_rating:.0f}%")
    elif final_rating >= 40:
        print(f"\n⚠️  MODERATE theme matching: {final_rating:.0f}%")
    else:
        print(f"\n❌ POOR theme matching: {final_rating:.0f}%")

WHITE EQUIPMENT THEME ANALYSIS:
White Equipment Theme Configuration:
Strategy: Equipment-based deck with efficient creatures and powerful gear
Keywords: ['equipment', 'attach', 'equipped', 'equip', 'metalcraft', 'artifact', 'power', 'toughness', 'sword', 'blade', 'equipped creature', 'gets +', 'artifact creature', 'living weapon', 'armor']
Archetype: Midrange

ACTUAL WHITE EQUIPMENT DECK (13 cards):
----------------------------------------
Equipment: 3
Creatures: 9
Other spells: 3

EQUIPMENT IN DECK:
  ⚔️ Mandibular Kite           | CMC: 1  | Creature Artifact - Equipment
  ⚔️ Ancestral Blade           | CMC: 2  | Creature Artifact - Equipment
  ⚔️ Greatsword of Tyr         | CMC: 2  | Artifact - Equipment

CREATURES IN DECK:
  🧙 Mandibular Kite           | CMC: 1  | nan/nan | Creature Artifact - Equipment
  🧙 Ancestral Blade           | CMC: 2  | nan/nan | Creature Artifact - Equipment
  🧙 Faerie Guidemother        | CMC: 1  | 1.0/1.0 | Creature - Faerie 
  🧙 Novice Inspector         

In [10]:
# ANALYSIS: Improvements needed for score_card_for_theme function
print("SCORE_CARD_FOR_THEME IMPROVEMENT ANALYSIS")
print("=" * 50)

# Current White Equipment theme configuration
white_equipment_config = ALL_THEMES['White Equipment']
print("White Equipment Theme:")
print(f"Strategy: {white_equipment_config['strategy']}")
print(f"Keywords: {white_equipment_config['keywords']}")
print(f"Archetype: {white_equipment_config['archetype']}")

print(f"\nCURRENT ISSUES IDENTIFIED:")
print("-" * 30)
print("1. ❌ MANA CURVE MISMATCH:")
print("   • Theme configured as 'Midrange' (expects CMC 2.5-4.0)")
print("   • Actual deck has 1.8 average CMC (too aggressive)")
print("   • Need Midrange archetype bonuses for 2-4 CMC cards")

print("\n2. ⚠️  MISSING ARCHETYPE HANDLING:")
print("   • Current function only handles 'Aggro' and 'Control'")
print("   • 'Midrange' archetype has no mana curve bonuses")
print("   • Need to add Midrange-specific scoring")

print("\n3. 🔧 EQUIPMENT SYNERGY GAPS:")
print("   • Equipment theme needs creature quality scoring")
print("   • Should favor creatures that benefit from equipment")
print("   • Need to check for evasion, low toughness, etc.")

print(f"\nPROPOSED IMPROVEMENTS:")
print("-" * 30)

print("A. ADD MIDRANGE ARCHETYPE SUPPORT:")
print("   • Bonus for CMC 2-4 cards in Midrange themes")
print("   • Penalty for CMC 1 and CMC 5+ in Midrange")
print("   • Target average CMC of 2.5-4.0")

print("\nB. ENHANCE EQUIPMENT-SPECIFIC SCORING:")
print("   • Bonus for creatures with evasion (flying, shadow, etc.)")
print("   • Bonus for creatures with low toughness (good equipment carriers)")
print("   • Bonus for creatures with triggered abilities on attack/damage")
print("   • Penalty for creatures with 0 power (can't benefit from +power equipment)")

print("\nC. IMPROVE ARTIFACT SYNERGY:")
print("   • Higher bonus for Equipment cards specifically")
print("   • Bonus for 'Living Weapon' equipment")
print("   • Bonus for artifact creatures in artifact themes")

print("\nD. STRATEGY-SPECIFIC BALANCING:")
print("   • Equipment decks need mix of cheap equipment + efficient creatures")
print("   • Avoid too many expensive equipment in same deck")
print("   • Ensure equipment-to-creature ratio is reasonable")

print(f"\nCODE CHANGES NEEDED:")
print("-" * 25)

print("1. Add Midrange archetype case:")
print("   elif archetype == 'Midrange':")
print("       if 2 <= cmc <= 4: score += 1.0")
print("       elif cmc == 1: score -= 0.5")
print("       elif cmc >= 5: score -= 1.0")

print("\n2. Add equipment-specific creature scoring:")
print("   if 'equipment' in theme_config['keywords'] and 'creature' in card_type:")
print("       # Check for evasion keywords")
print("       if any(kw in oracle_text for kw in ['flying', 'shadow', 'unblockable']):")
print("           score += 1.0")
print("       # Bonus for efficient equipment carriers")
print("       if toughness > 0 and toughness <= 3: score += 0.5")

print("\n3. Equipment type bonuses:")
print("   if 'equipment' in card_type:")
print("       score += 2.0  # Strong bonus for actual equipment")
print("       if 'living weapon' in oracle_text: score += 1.0")

# Let's check what creatures are in White Equipment to validate our analysis
print(f"\nVALIDATION: Current White Equipment creatures for improvement analysis")
print("-" * 60)
white_equipment_deck = deck_dataframes.get('White Equipment', pd.DataFrame())
if not white_equipment_deck.empty:
    creatures = white_equipment_deck[white_equipment_deck['Type'].str.contains('Creature', case=False, na=False)]
    
    for _, creature in creatures.iterrows():
        cmc = creature.get('CMC', '?')
        power = creature.get('Power', '?')
        toughness = creature.get('Toughness', '?')
        oracle_text = str(creature.get('Oracle Text', '')).lower()
        
        # Check for equipment synergy indicators
        synergy_indicators = []
        if 'flying' in oracle_text: synergy_indicators.append('Flying')
        if 'shadow' in oracle_text: synergy_indicators.append('Shadow')  
        if 'unblockable' in oracle_text: synergy_indicators.append('Unblockable')
        if 'first strike' in oracle_text: synergy_indicators.append('First Strike')
        if 'double strike' in oracle_text: synergy_indicators.append('Double Strike')
        
        synergy_str = f" [{', '.join(synergy_indicators)}]" if synergy_indicators else " [No evasion]"
        print(f"  {creature['name']:<25} | CMC:{cmc} | {power}/{toughness}{synergy_str}")
        
        # Check if this would benefit from our proposed improvements
        improvements = []
        if cmc >= 2 and cmc <= 4: improvements.append("✅ Midrange CMC")
        elif cmc == 1: improvements.append("❌ Too cheap for Midrange")
        elif cmc >= 5: improvements.append("❌ Too expensive for Midrange")
        
        if synergy_indicators: improvements.append("✅ Good equipment carrier")
        elif power != '?' and power > 0: improvements.append("⚠️  Okay carrier")
        else: improvements.append("❌ Poor equipment carrier")
        
        if improvements:
            print(f"    → Improvements would rate: {' | '.join(improvements)}")

print(f"\n💡 EXPECTED IMPACT:")
print("   • Better mana curve for Midrange themes")
print("   • More synergistic creature selection")
print("   • Higher equipment density")
print("   • More coherent midrange strategy execution")

SCORE_CARD_FOR_THEME IMPROVEMENT ANALYSIS
White Equipment Theme:
Strategy: Equipment-based deck with efficient creatures and powerful gear
Keywords: ['equipment', 'attach', 'equipped', 'equip', 'metalcraft', 'artifact', 'power', 'toughness', 'sword', 'blade', 'equipped creature', 'gets +', 'artifact creature', 'living weapon', 'armor']
Archetype: Midrange

CURRENT ISSUES IDENTIFIED:
------------------------------
1. ❌ MANA CURVE MISMATCH:
   • Theme configured as 'Midrange' (expects CMC 2.5-4.0)
   • Actual deck has 1.8 average CMC (too aggressive)
   • Need Midrange archetype bonuses for 2-4 CMC cards

2. ⚠️  MISSING ARCHETYPE HANDLING:
   • Current function only handles 'Aggro' and 'Control'
   • 'Midrange' archetype has no mana curve bonuses
   • Need to add Midrange-specific scoring

3. 🔧 EQUIPMENT SYNERGY GAPS:
   • Equipment theme needs creature quality scoring
   • Should favor creatures that benefit from equipment
   • Need to check for evasion, low toughness, etc.

PROPOSED IM

In [11]:
# Test improved scoring function (simulated improvements)
from src.construct import score_card_for_theme
import pandas as pd
import re

def score_card_for_theme_improved(card: pd.Series, theme_config: dict) -> float:
    """Improved version with Midrange and Equipment enhancements."""
    keywords = theme_config['keywords']
    score = 0.0
    
    # Get card properties
    cmc = card.get('CMC', 0) if pd.notna(card.get('CMC', 0)) else 0
    power = card.get('Power', 0) if pd.notna(card.get('Power', 0)) else 0
    toughness = card.get('Toughness', 0) if pd.notna(card.get('Toughness', 0)) else 0
    archetype = theme_config.get('archetype', '')
    
    # Get text fields for searching
    oracle_text = str(card['Oracle Text']).lower() if pd.notna(card['Oracle Text']) else ""
    card_type = str(card['Type']).lower() if pd.notna(card['Type']) else ""
    card_name = str(card['name']).lower() if pd.notna(card['name']) else ""
    
    # Combine all text for searching
    searchable_text = f"{oracle_text} {card_type} {card_name}"
    
    # Score based on keyword matches
    for keyword in keywords:
        keyword_lower = keyword.lower()
        if keyword_lower in searchable_text:
            if re.search(r'\b' + re.escape(keyword_lower) + r'\b', searchable_text):
                score += 1.0
            elif keyword_lower in searchable_text:
                score += 0.5
    
    # IMPROVED MANA CURVE SCORING
    if archetype == 'Aggro':
        if cmc == 1:
            score += 2.0
        elif cmc == 2:
            score += 1.0
        elif cmc >= 4:
            score -= 1.0
    elif archetype == 'Control':
        if cmc >= 4:
            score += 0.5
        elif cmc == 1:
            score -= 0.5
    elif archetype == 'Midrange':  # NEW!
        if 2 <= cmc <= 4:
            score += 1.0  # Bonus for 2-4 CMC in midrange
        elif cmc == 1:
            score -= 0.5  # Penalty for too aggressive
        elif cmc >= 5:
            score -= 1.0  # Penalty for too expensive
    
    # Original keyword-specific scoring
    if 'cheap' in keywords and cmc <= 2:
        score += 1.0
    if 'efficient' in keywords and 'creature' in card_type and power > 0 and cmc > 0:
        if power >= cmc:
            score += 1.0
        elif power >= cmc - 1:
            score += 0.5
    if 'low cost' in keywords and cmc <= 2:
        score += 1.0
    if 'small' in keywords and 'creature' in card_type and power <= 2:
        score += 0.5
    
    # IMPROVED ARTIFACT/EQUIPMENT SCORING
    if 'artifact' in card_type:
        if any(kw in theme_config['keywords'] for kw in ['artifact', 'equipment', 'metalcraft']):
            score += 0.5
        
        # NEW: Special equipment bonuses
        if 'equipment' in card_type:
            score += 2.0  # Strong bonus for actual equipment
            if 'living weapon' in oracle_text:
                score += 1.0
    
    # NEW: Equipment-specific creature scoring
    if 'equipment' in theme_config['keywords'] and 'creature' in card_type:
        # Evasion bonus
        evasion_keywords = ['flying', 'shadow', 'unblockable', 'menace', 'trample']
        if any(kw in oracle_text for kw in evasion_keywords):
            score += 1.0
        
        # Combat abilities bonus
        combat_keywords = ['first strike', 'double strike', 'vigilance', 'lifelink']
        if any(kw in oracle_text for kw in combat_keywords):
            score += 0.5
        
        # Good equipment carrier bonus
        if power > 0 and toughness > 0 and toughness <= 3:
            score += 0.5
        
        # Penalty for non-attackers
        if power == 0 or 'defender' in oracle_text:
            score -= 1.0
    
    # Type-based bonuses
    if 'creature' in card_type and any(kw in theme_config['keywords'] for kw in ['creature', 'tribal', 'aggressive']):
        score += 0.3
    if ('instant' in card_type or 'sorcery' in card_type) and any(kw in theme_config['keywords'] for kw in ['instant', 'sorcery', 'spells', 'burn', 'counter']):
        score += 0.3
    
    return score

print("TESTING IMPROVED SCORING ON WHITE EQUIPMENT DECK")
print("=" * 55)

white_equipment_config = ALL_THEMES['White Equipment']
white_equipment_deck = deck_dataframes.get('White Equipment', pd.DataFrame())

if not white_equipment_deck.empty:
    print(f"Comparing OLD vs NEW scores for each card:\n")
    
    total_old_score = 0
    total_new_score = 0
    
    for _, card in white_equipment_deck.iterrows():
        old_score = score_card_for_theme(card, white_equipment_config)
        new_score = score_card_for_theme_improved(card, white_equipment_config)
        
        total_old_score += old_score
        total_new_score += new_score
        
        # Calculate improvement
        improvement = new_score - old_score
        if improvement > 0:
            arrow = "📈"
        elif improvement < 0:
            arrow = "📉"
        else:
            arrow = "➡️"
        
        cmc = card.get('CMC', '?')
        card_type_short = card['Type'].split(' -')[0] if ' -' in card['Type'] else card['Type']
        
        print(f"{card['name']:<25} | CMC:{cmc} | {card_type_short:<12}")
        print(f"  Old: {old_score:4.1f} → New: {new_score:4.1f} ({improvement:+4.1f}) {arrow}")
        
        # Explain the improvement
        if 'equipment' in card['Type'].lower():
            print(f"    💡 Equipment bonus: +2.0 points")
        if 'creature' in card['Type'].lower() and 2 <= cmc <= 4:
            print(f"    💡 Midrange CMC bonus: +1.0 points")
        if 'creature' in card['Type'].lower() and cmc == 1:
            print(f"    ⚠️  Midrange CMC penalty: -0.5 points (too aggressive)")
        
        print()
    
    avg_old = total_old_score / len(white_equipment_deck)
    avg_new = total_new_score / len(white_equipment_deck)
    
    print(f"SUMMARY:")
    print(f"Average OLD score: {avg_old:.2f}")
    print(f"Average NEW score: {avg_new:.2f}")
    print(f"Improvement: {avg_new - avg_old:+.2f} points per card")
    
    print(f"\nEXPECTED IMPACT:")
    print("✅ Equipment cards now score much higher")
    print("✅ Midrange CMC cards (2-4) get bonuses") 
    print("✅ 1-CMC cards get penalties in Midrange themes")
    print("✅ Better equipment carrier selection")

TESTING IMPROVED SCORING ON WHITE EQUIPMENT DECK
Comparing OLD vs NEW scores for each card:

Mandibular Kite           | CMC:1 | Creature Artifact
  Old:  8.5 → New: 11.0 (+2.5) 📈
    💡 Equipment bonus: +2.0 points
    ⚠️  Midrange CMC penalty: -0.5 points (too aggressive)

Ancestral Blade           | CMC:2 | Creature Artifact
  Old:  8.5 → New: 10.5 (+2.0) 📈
    💡 Equipment bonus: +2.0 points
    💡 Midrange CMC bonus: +1.0 points

Greatsword of Tyr         | CMC:2 | Artifact    
  Old:  7.0 → New: 10.0 (+3.0) 📈
    💡 Equipment bonus: +2.0 points

Faerie Guidemother        | CMC:1 | Creature    
  Old:  1.0 → New:  2.0 (+1.0) 📈
    ⚠️  Midrange CMC penalty: -0.5 points (too aggressive)

Novice Inspector          | CMC:1 | Creature    
  Old:  1.0 → New:  1.0 (+0.0) ➡️
    ⚠️  Midrange CMC penalty: -0.5 points (too aggressive)

Thraben Inspector         | CMC:1 | Creature    
  Old:  1.0 → New:  1.0 (+0.0) ➡️
    ⚠️  Midrange CMC penalty: -0.5 points (too aggressive)

Cathar Commando   